In [1]:
import os
import sys
import glob
import keras
from keras.models import Sequential
from tensorflow.keras.utils import img_to_array, array_to_img, load_img
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import Conv2D
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import SGD
from keras.callbacks import LearningRateScheduler
from keras import backend as K
K.set_image_data_format('channels_first')

In [60]:
# from PIL import Image

# widths = []
# heights = []

# for dir in os.listdir(train_dir):
#     dir_path = train_dir + "/" + dir
#     if os.path.isdir(dir_path):
#         for img in os.listdir(dir_path):
#             img_path = os.path.join(train_dir + "/" + dir, img) # Making image file path
#             im = Image.open(img_path)
#             widths.append(im.size[0])
#             heights.append(im.size[1])

# AVG_HEIGHT = round(sum(heights)/len(heights))
# AVG_WIDTH = round(sum(widths)/len(widths))

In [61]:
AVG_HEIGHT

476

In [62]:
AVG_WIDTH

629

In [71]:
# default settings
img_width, img_height = 224, 224

train_dir = '/Users/Janeshelbyporter/Documents/OneDrive-CofC/BE/Image-Recognition-with-Deep-Learning-master/data/butterflies/train_01'
train_dir2 = '/Users/Janeshelbyporter/Documents/OneDrive-CofC/BE/Image-Recognition-with-Deep-Learning-master/data/butterflies/train_02'
validate_dir = '/Users/Janeshelbyporter/Documents/OneDrive-CofC/BE/Image-Recognition-with-Deep-Learning-master/data/butterflies/validate'
epochs = 5
batch_size = 32
num_classes = 10

In [72]:
# get number of images in training directory
nb_train_samples = 0
for r, dirs, files in os.walk(train_dir):
    for dr in dirs:
        nb_train_samples += len(glob.glob(os.path.join(r, dr + "/*")))
print(nb_train_samples)

592


In [73]:
# get number of images in validation directory
nb_validate_samples = 0
for r, dirs, files in os.walk(validate_dir):
    for dr in dirs:
        nb_validate_samples += len(glob.glob(os.path.join(r, dr + "/*")))
print(nb_validate_samples)

169


In [74]:
# data pre-processing for training
train_datagen =  ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip = True)

In [75]:
# data pre-processing for validation
validate_datagen =  ImageDataGenerator(
    rescale = 1./255,
    rotation_range = 20,
    width_shift_range = 0.2,
    height_shift_range = 0.2,
    shear_range = 0.2,
    zoom_range = 0.2,
    fill_mode = 'nearest',
    horizontal_flip = True)

In [76]:
# generate and store training data
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size)

Found 592 images belonging to 10 classes.


In [77]:
# generate and store validation data
validate_generator = validate_datagen.flow_from_directory(
    validate_dir,
    target_size = (img_width, img_height),
    batch_size = batch_size)

Found 169 images belonging to 10 classes.


In [78]:
# define the model
def convnet_model():
    model = Sequential()

    model.add(Conv2D(32, (3, 3), padding='same',
                    input_shape=(3, img_width, img_height),
                    activation='relu'))
    model.add(Conv2D(32, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(64, (3, 3), padding='same',
                    activation='relu'))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Conv2D(128, (3, 3), padding='same',
                    activation='relu'))
    model.add(Conv2D(128, (3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))

    model.add(Flatten())
    model.add(Dense(512, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [79]:
# define learning rate schedule
def lr_schedule(epoch):
    return lr * (0.1 ** int(epoch / 10))

In [80]:
# instantiate CNN model
model_final = convnet_model()

In [81]:
# Train the model
lr = 0.01
sgd = SGD(learning_rate=lr, decay=1e-6, momentum=0.9, nesterov=True)
model_final.compile(loss='categorical_crossentropy',
              optimizer=sgd,
              metrics=['accuracy'])

In [82]:
model_final.fit(
    train_generator,
    epochs = epochs,
# #     steps_per_epoch = nb_train_samples / batch_size,
    validation_data = validate_generator,
#     validation_steps = nb_validate_samples / batch_size,
#     class_weight='auto',
    callbacks=[LearningRateScheduler(lr_schedule)])

Epoch 1/5


2022-10-18 00:59:58.418662: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - ETA: 0s - loss: 2.3058 - accuracy: 0.1115

2022-10-18 01:00:18.335219: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.


19/19 [==============================] - 22s 1s/step - loss: 2.3058 - accuracy: 0.1115 - val_loss: 2.2938 - val_accuracy: 0.1183 - lr: 0.0100
Epoch 2/5
19/19 [==============================] - 21s 1s/step - loss: 2.2892 - accuracy: 0.1318 - val_loss: 2.2826 - val_accuracy: 0.1775 - lr: 0.0100
Epoch 3/5
19/19 [==============================] - 21s 1s/step - loss: 2.2590 - accuracy: 0.1689 - val_loss: 2.2462 - val_accuracy: 0.2130 - lr: 0.0100
Epoch 4/5
19/19 [==============================] - 21s 1s/step - loss: 2.2032 - accuracy: 0.2027 - val_loss: 2.2892 - val_accuracy: 0.1243 - lr: 0.0100
Epoch 5/5
19/19 [==============================] - 21s 1s/step - loss: 2.1616 - accuracy: 0.1824 - val_loss: 2.1395 - val_accuracy: 0.2426 - lr: 0.0100
